## Loading the image data

In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import h5py

from sklearn.datasets import load_files
from sklearn.metrics import mean_absolute_error, median_absolute_error

from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as preprocess_input_xception
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D,MaxPooling1D, Dropout, Flatten, Dense
from keras import layers
from keras import models
from keras.activations import selu
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.metrics import mean_absolute_error as mean_absolute_error_keras

In [2]:
bone_names = [int(item[25:-4]) for item in sorted(glob("boneage-training-dataset/*"))]

## Creating a dataframe consisting of ids, gender, image paths and bone ages

In [3]:
bone_age_df = pd.read_csv('boneage-training-dataset.csv')
image_paths = []
for i in sorted(bone_names):
    image_paths.append("boneage-training-dataset/{}.png".format(str(i)))
bone_age_df['img_paths'] = image_paths
bone_age_df['gender'] = bone_age_df['male'].map(lambda x: 0 if x else 1)
bone_age_df.drop('male',axis=1)
bone_age_df.head()

,id,boneage,male,img_paths,gender
0,1377,180,False,boneage-training-dataset/1377.png,1
1,1378,12,False,boneage-training-dataset/1378.png,1
2,1379,94,False,boneage-training-dataset/1379.png,1
3,1380,120,True,boneage-training-dataset/1380.png,0
4,1381,82,False,boneage-training-dataset/1381.png,1


## Dividing the frame into training, validation and test sets

Since the test csv file did not contain the bone age columns, so I had to divide the training data into three parts

Train size = 11000

Valid size = 1400

test size = 211

In [4]:
train_df = bone_age_df[1:11000]
valid_df = bone_age_df[11000:12400]
test_df = bone_age_df[12400:12611]

## Loading Target ages

In [5]:
train_targets = train_df['boneage']
valid_targets = valid_df['boneage']
test_targets = test_df['boneage']

# Transfer Learning

## Loading the features for Xception model

In [6]:
train_xception = np.load("BoneFeaturesXceptionTrain.npz")['train']
valid_xception = np.load("BoneFeaturesXceptionValid.npz")['valid']
test_xception = np.load("BoneFeaturesXceptionTest.npz")['test']

## Building the model architecture

In [7]:
def model_architecture(train_data,name):
    #Creating Xception path of the network
    xception_input = Input(shape=train_data.shape[1:], name=name)
    xception_pool = GlobalAveragePooling2D()(xception_input)
    xception_dense = Dense(128,activation='selu')(xception_pool)
    xception_dense = Dropout(0.25)(xception_dense)
    
    #Creating gender path of the network
    gender_input = Input(shape=(1,), name='gender')
    gender_part = Dense(128, activation='selu')(gender_input)
    
    #marging the two paths - Xception and Gender
    net = layers.Add()([xception_dense, gender_part])

    #Adding further Dense layers
    net = Dense(128, activation='selu')(net)
    net = Dropout(0.25)(net)
    net = Dense(256, activation='selu')(net)
    net = Dropout(0.25)(net)
    net = Dense(128, activation='selu')(net)
    
    #final output layer with one neuron to predict the bone age
    bone_age_layer = Dense(units=1, name='age_values')(net)

    bone_model = models.Model(inputs=[xception_input, gender_input], outputs=bone_age_layer)

    return bone_model

# Creating model based on Xception
bone_model_xception = model_architecture(train_xception,'xception')
print(bone_model_xception.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
xception (InputLayer)           [(None, 7, 7, 2048)] 0                                            
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 2048)         0           xception[0][0]                   
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          262272      global_average_pooling2d[0][0]   
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
______________________________________________________________________________________________

## Compiling the model

In [8]:
bone_model_xception.compile(optimizer = 'adam', loss = 'mse', metrics = [mean_absolute_error_keras])

## Loading the best weights and training the model

In [9]:
checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

bone_model_xception.fit([train_xception,train_df['gender']], train_targets, 
          validation_data=([valid_xception,valid_df['gender']], valid_targets),
          epochs=40, batch_size=50, callbacks=[checkpointer], verbose=1)

Epoch 1/40
217/220 [============================>.] - ETA: 0s - loss: 1711.6454 - mean_absolute_error: 31.0901
Epoch 00001: val_loss improved from inf to 555.16284, saving model to weights.best.from_scratch.hdf5
220/220 [==============================] - 3s 14ms/step - loss: 1698.1010 - mean_absolute_error: 30.9513 - val_loss: 555.1628 - val_mean_absolute_error: 18.6968
Epoch 2/40
216/220 [============================>.] - ETA: 0s - loss: 607.7294 - mean_absolute_error: 19.3639
Epoch 00002: val_loss improved from 555.16284 to 375.57126, saving model to weights.best.from_scratch.hdf5
220/220 [==============================] - 3s 13ms/step - loss: 607.4499 - mean_absolute_error: 19.3613 - val_loss: 375.5713 - val_mean_absolute_error: 15.2345
Epoch 3/40
217/220 [============================>.] - ETA: 0s - loss: 502.8750 - mean_absolute_error: 17.4972
Epoch 00003: val_loss improved from 375.57126 to 347.53699, saving model to weights.best.from_scratch.hdf5
220/220 [========================

218/220 [============================>.] - ETA: 0s - loss: 264.4595 - mean_absolute_error: 12.6967
Epoch 00024: val_loss improved from 259.30341 to 252.63966, saving model to weights.best.from_scratch.hdf5
220/220 [==============================] - 3s 14ms/step - loss: 264.7074 - mean_absolute_error: 12.7065 - val_loss: 252.6397 - val_mean_absolute_error: 12.1001
Epoch 25/40
218/220 [============================>.] - ETA: 0s - loss: 263.3037 - mean_absolute_error: 12.6337
Epoch 00025: val_loss did not improve from 252.63966
220/220 [==============================] - 3s 14ms/step - loss: 263.3634 - mean_absolute_error: 12.6307 - val_loss: 272.7971 - val_mean_absolute_error: 12.6300
Epoch 26/40
216/220 [============================>.] - ETA: 0s - loss: 270.2253 - mean_absolute_error: 12.8472
Epoch 00026: val_loss did not improve from 252.63966
220/220 [==============================] - 3s 13ms/step - loss: 269.9561 - mean_absolute_error: 12.8413 - val_loss: 277.9416 - val_mean_absolute_e

## Making the predictions and evaluating the model

In [10]:
predictions = []
    #predictions = [(bone_model.predict([np.expand_dims(feature_1, axis=0),np.expand_dims(feature_2, axis=0)])) for feature_1,feature_2 in zip(test_xception,test['male'])]
for i,j in zip(test_xception,test_df['gender']):
    predictions.append(bone_model_xception.predict([np.expand_dims(i,axis=0),np.expand_dims(j,axis=0)])[0][0])
targets = []
for i in test_targets:
    targets.append(i)
    
print("Mean:",mean_absolute_error(predictions,targets))
print("Median:",median_absolute_error(predictions,targets))

Mean: 11.33025228129744
Median: 9.334098815917969


In [11]:
output = list(zip(test_df['gender'],targets,predictions))
df = pd.DataFrame(output, columns=['Male','Actual Target','Predicted_target'])
df.to_csv("Final results.csv")